In [1]:
import vnstock

In [12]:
from vnstock import Listing
listing = Listing(source='VCI')
df1 = listing.all_symbols()
df2 = listing.symbols_by_industries()
df1.head()


,symbol,organ_name
0,YTC,Công ty Cổ phần Xuất nhập khẩu Y tế Thành phố ...
1,YEG,Công ty Cổ phần Tập đoàn Yeah1
2,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái
3,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái
4,XPH,Công ty Cổ phần Xà phòng Hà Nội


In [13]:
df2.head()

,symbol,organ_name,icb_name3,icb_name2,icb_name4,com_type_code,icb_code1,icb_code2,icb_code3,icb_code4
0,BQP,Công ty Cổ phần Nhựa chất lượng cao Bình Thuận,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",CT,1000,1300,1350,1353
1,F88,Công ty Cổ phần Đầu tư F88,Dịch vụ tài chính,Dịch vụ tài chính,Tài chính cá nhân,CT,8000,8700,8770,8773
2,DVT,Trung tâm Đào tạo nghiệp vụ Giao thông vận tải...,Tư vấn & Hỗ trợ Kinh doanh,Hàng & Dịch vụ Công nghiệp,Đào tạo & Việc làm,CT,2000,2700,2790,2793
3,AGR,Công ty Cổ phần Chứng khoán AGRIBANK,Dịch vụ tài chính,Dịch vụ tài chính,Môi giới chứng khoán,CK,8000,8700,8770,8777
4,ABS,Công ty Cổ phần Dịch vụ Nông nghiệp Bình Thuận,Hóa chất,Hóa chất,"Sản phẩm hóa dầu, Nông dược & Hóa chất khác",CT,1000,1300,1350,1357


In [16]:
# merge df1 and df2 on 'symbol' (change how='inner' to 'left' or 'outer' if needed)
df3 = df1.merge(df2, on='symbol', how='inner', suffixes=('_df1', '_df2'))

# If both frames have 'organ_name', keep the one from df2 when available, otherwise fallback to df1
if 'organ_name_df1' in df3.columns and 'organ_name_df2' in df3.columns:
    df3['organ_name'] = df3['organ_name_df2'].combine_first(df3['organ_name_df1'])
    df3 = df3.drop(columns=['organ_name_df1', 'organ_name_df2'])
# move 'organ_name' to be the 2nd column (right after 'symbol')
if 'organ_name' in df3.columns and df3.columns.tolist().index('organ_name') != 1:
    cols = df3.columns.tolist()
    cols.insert(1, cols.pop(cols.index('organ_name')))
    df3 = df3[cols]

df3.head()

# inspect result
df3.head()

,symbol,organ_name,icb_name3,icb_name2,icb_name4,com_type_code,icb_code1,icb_code2,icb_code3,icb_code4
0,YTC,Công ty Cổ phần Xuất nhập khẩu Y tế Thành phố ...,Thiết bị và Dịch vụ Y tế,Y tế,Dụng cụ y tế,CT,4000,4500,4530,4537
1,YEG,Công ty Cổ phần Tập đoàn Yeah1,Truyền thông,Truyền thông,Giải trí & Truyền thông,CT,5000,5500,5550,5553
2,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái,Khai khoáng,Tài nguyên Cơ bản,Khai khoáng,CT,1000,1700,1770,1775
3,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái,Xây dựng và Vật liệu,Xây dựng và Vật liệu,Vật liệu xây dựng & Nội thất,CT,2000,2300,2350,2353
4,XPH,Công ty Cổ phần Xà phòng Hà Nội,Hàng cá nhân,Hàng cá nhân & Gia dụng,Hàng cá nhân,CT,3000,3700,3760,3767


In [17]:
from pathlib import Path

out_dir = Path('backend/data')
out_dir.mkdir(parents=True, exist_ok=True)

# Save symbol and organ_name from df3 to JSON (adjust columns if you want full df3)
df3[['symbol', 'organ_name']].to_json(out_dir / 'organ_name.json', orient='records', force_ascii=False)

print(f"Saved {len(df3)} records to {out_dir / 'organ_name.json'}")

Saved 1582 records to backend/data/organ_name.json
